## Model Evaluating, Part 3

- Measure a model's relevance, with using precision, recall, and the F1-score
- Reading a confusion matrix
- Optimizing the precision and recall of a classification model 
- Plotting a receiver operating characteristic (ROC curve)